In [7]:
# Imports
import pandas as pd
import numpy as np
import sys

from datetime import datetime as dt

# Scikit-learn
from sklearn.model_selection import *
from sklearn.ensemble import *
from sklearn.metrics import *
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from lightgbm import LGBMClassifier
from imblearn.combine import *
from imblearn.over_sampling import *
from imblearn.under_sampling import *
from imblearn.pipeline import make_pipeline
from sklearn.preprocessing import *
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import *
from sklearn.svm import *
from scipy.stats import *

import missingno as msno
import prince

import seaborn as sns
import matplotlib.pyplot as plt

In [13]:
X = pd.read_csv('X_2_dimreduce-2023-05-14.csv')
y = pd.read_csv('train.csv')['DC201']
X_test = pd.read_csv('X_test_2_dimreduce-2023-05-14.csv')

In [14]:
y = [1 if i == 'Layak Minum' else 0 for i in y]

In [16]:
clf1 = SVC(probability=True)
clf2 = CalibratedClassifierCV(LGBMClassifier(boosting_type='dart',
                                colsample_bytree=0.5003058449328365,
                                learning_rate=0.02370660486501326, max_depth=5,
                                min_child_samples=78,
                                min_child_weight=8.40380991588707,
                                min_split_gain=0.9210732111954107,
                                n_estimators=123, num_leaves=642,
                                random_state=4244, reg_alpha=7.9257571186733795,
                                reg_lambda=3.29115219914055,
                                scale_pos_weight=1.8862036127575263,
                                subsample=0.8772357357787093,
                                subsample_freq=1),
                                cv = StratifiedKFold(5))
clf3 = CalibratedClassifierCV(SGDClassifier())
clf4 = RandomForestClassifier()
clf5 = CalibratedClassifierCV(LGBMClassifier(boosting_type='dart',
                                colsample_bytree=0.4577758036149866,
                                learning_rate=0.029440262622770587,
                                max_depth=20, min_child_samples=85,
                                min_child_weight=7.678753061525662,
                                min_split_gain=0.9867940295277046,
                                n_estimators=112, num_leaves=353,
                                random_state=42444,
                                reg_alpha=4.9751408654624685,
                                reg_lambda=1.3254075005190202,
                                scale_pos_weight=1.8097591084310098,
                                subsample=0.9475980584116197,
                                subsample_freq=9),
                                cv = StratifiedKFold(5))

In [17]:
combine = VotingClassifier(
    estimators = 
    [
        ('svm', clf1),
        ('lgbm1', clf2),
        ('sgd', clf3),
        ('rf', clf4),
        ('lgbm2', clf5)
    ],
    voting = 'soft',
    weights = [0.20, 0.25, 0.15, 0.15, 0.25],
    n_jobs = -1,
    verbose = 2
)
combine.fit(X, y)
y_pred = combine.predict(X_test)
y_pred = ['Layak Minum' if i == 1 else 'Tidak Layak Minum' for i in y_pred]

[Voting] ...................... (3 of 5) Processing sgd, total=   0.5s
[Voting] .................... (2 of 5) Processing lgbm1, total=   5.8s
[Voting] .................... (5 of 5) Processing lgbm2, total=   8.7s
[Voting] ....................... (4 of 5) Processing rf, total=  16.6s
[Voting] ...................... (1 of 5) Processing svm, total= 2.8min


In [20]:
pd.DataFrame({
    'id' : pd.read_csv('test.csv')['id'],
    'DC201' : y_pred
}).to_csv('submission_voting_2023-05-14.csv', index=False)

In [21]:
pd.read_csv('submission_voting_2023-05-14.csv')['DC201'].value_counts()

Layak Minum          11856
Tidak Layak Minum      134
Name: DC201, dtype: int64

In [18]:
pd.read_csv('lgbm_ada2-2023-05-14submission.csv')['DC201'].value_counts()

Layak Minum          11772
Tidak Layak Minum      218
Name: DC201, dtype: int64